In [ ]:
import cv2
from ultralytics import YOLO
import cvzone
import json

In [ ]:
# Define the class
class_list = ['Staff']

# Video for testing
video = './videos/sample.mp4'

cam = cv2.VideoCapture(video2)

# Use YOLOv8l model that trained with custom dataset
# Custom dataset is prepared manually and labelled with class with LabeLIMG
model = YOLO('./best_4_yolo8l.pt')

# Record current frame and x and y coordination when the class appear in the screen.
current_frame = 0
x_y_coor = {}
while True:
    ret, vid = cam.read()
    results = model(vid, stream=True)
    
    for r in results:
        bboxes = r.boxes

        # Each box in a result
        for bbox in bboxes:
            x1, y1, x2, y2 = bbox.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            coor = ((x1+x2)//2, (y1+y2)//2)
            # Confidence score
            conf = round(bbox.conf[0].item(), 2)

            # Class id
            cls = int(bbox.cls[0].item())
            class_name = class_list[cls]

            if conf > 0.7:
                cvzone.putTextRect(vid, '{}:{}'.format(class_name, conf), (max(0, x1), max(20, y1 - 20)), scale=1,
                                   thickness=2, colorR=(255, 0, 0), offset=3)
                cvzone.cornerRect(vid, (x1, y1, w, h), l=15, t=3)
                cv2.putText(vid, str(coor), (coor[0], coor[1]), fontFace=cv2.FONT_HERSHEY_PLAIN,fontScale=1, color=(0,255,0), thickness=2)

                frame_name = f'frame/frame{str(current_frame)}.jpg'

                # Export the frame when the class appear in the screen.
                cv2.imwrite(frame_name, vid)
                x_y_coor[f'frame{current_frame}'] = coor
                current_frame += 1
    
    # Export the current frame into a text file
    with open('./frame/coor.txt','w') as f:
        f.write(json.dumps(x_y_coor))
        
    # Show the video
    cv2.imshow('video', vid)

    key = cv2.waitKey(1)
    if key == 27:
        break


cv2.destroyAllWindows()